In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import glob
import cv2
import torchvision.transforms as transforms

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True)
    )

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((1500,1500)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
class img_dataset(Dataset):
    def __init__(self, img_dir_motion_blurred, img_dir_sharp, transform=None):
        self.img_dir_motion = img_dir_motion_blurred
        self.img_dir_sharp = img_dir_sharp
        self.transform = transform
        self.img_files_jpg_motion = sorted(glob.glob(img_dir_motion_blurred+'/*jpg'), key=lambda x: int(x.split('/')[-1].split('_')[0]))
        self.img_files_JPG_motion = sorted(glob.glob(img_dir_motion_blurred+'/*JPG'), key=lambda x: int(x.split('/')[-1].split('_')[0]))

        self.img_files_jpeg_motion = sorted(glob.glob(img_dir_motion_blurred+'/*jpeg'), key=lambda x: int(x.split('/')[-1].split('_')[0]))
        self.img_files_motion = self.img_files_jpg_motion+self.img_files_jpeg_motion+self.img_files_JPG_motion
        self.img_files_jpg_sharp = sorted(glob.glob(img_dir_sharp+'/*jpg'), key=lambda x: int(x.split('/')[-1].split('_')[0]))
        self.img_files_JPG_sharp = sorted(glob.glob(img_dir_sharp+'/*JPG'), key=lambda x: int(x.split('/')[-1].split('_')[0]))

        self.img_files_jpeg_sharp = sorted(glob.glob(img_dir_sharp+'/*jpeg'), key=lambda x: int(x.split('/')[-1].split('_')[0]))
        self.img_files_sharp = self.img_files_jpg_sharp+self.img_files_jpeg_sharp+self.img_files_JPG_sharp
    def __len__(self):
        return len(self.img_files_motion)
    def __getitem__(self, idx):
#         images_jpg_motion = cv2.imread(self.img_files_jpg_motion)
#         images_jpeg_motion = cv2.imread(self.img_files_jpeg_motion)
#         images_jpg_sharp = cv2.imread(self.img_files_jpg_sharp)
#         images_jpeg_sharp = cv2.imread(self.img_files_jpeg_sharp)
        images_motion = cv2.imread(self.img_files_motion[idx])
        images_sharp = cv2.imread(self.img_files_sharp[idx])
        if self.transform:
#                     images_jpg_motion = self.transform(images_jpg_motion)
#                     images_jpeg_motion = self.transform(images_jpeg_motion)
#                     images_jpg_sharp = self.ransform(images_jpg_sharp)
#                     images_jpeg_sharp = self.transform(images_jpeg_sharp)
            images_motion = self.transform(images_motion)
            images_sharp = self.transform(images_sharp)
        return images_motion, images_sharp

In [ ]:
images_dataset = img_dataset('/kaggle/input/blur-dataset/motion_blurred', '/kaggle/input/blur-dataset/sharp',transform)

In [ ]:
first_image1,first_image2  = img_dataset.__getitem__(images_dataset, 210)
print(first_image1.shape, first_image2.shape)

In [ ]:
len(images_dataset)

In [ ]:
len(images_dataset.img_files_sharp)

In [ ]:
len(images_dataset.img_files_motion)

In [ ]:
train_size = int(0.8* len(images_dataset))
test_size = len(images_dataset)-train_size

In [ ]:
train_dataset,test_dataset = random_split(images_dataset,[train_size,test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 1,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = 1,
                                           shuffle = True)

In [ ]:
# Cropping function to ensure matching sizes
def crop_tensor(tensor, target_tensor):
    target_size = target_tensor.size(2)
    tensor_size = tensor.size(2)
    delta = tensor_size - target_size
    delta = delta // 2
    return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]

In [ ]:
class convnn(nn.Module):
    def __init__(self):
        super(convnn, self).__init__()
        self.layer1 = double_conv(3, 64)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer2 = double_conv(64, 128)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer3 = double_conv(128, 256)
        self.max_pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer4 = double_conv(256, 512)
        self.max_pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer5 = double_conv(512, 1024)
        
        self.up1 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2)
        self.layer6 = double_conv(1024, 512)  # 512 (from upsampled layer5) + 512 (from layer4)

        self.up2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2)
        self.layer7 = double_conv(512, 256)  # 256 (from upsampled layer6) + 256 (from layer3)
        
        self.up3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2)
        self.layer8 = double_conv(256, 128)  # 128 (from upsampled layer7) + 128 (from layer2)
        
        self.up4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2)
        self.layer9 = double_conv(128, 64)  # 64 (from upsampled layer8) + 64 (from layer1)
        
        self.layer10 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=1)

    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.max_pool1(out1)

        out3 = self.layer2(out2)
        out4 = self.max_pool2(out3)

        out5 = self.layer3(out4)
        out6 = self.max_pool3(out5)

        out7 = self.layer4(out6)
        out8 = self.max_pool4(out7)

        out9 = self.layer5(out8)
        
        out10 = self.up1(out9)
        out10 = crop_tensor(out7, out10)
        out10 = torch.cat((out10, out7), dim=1)
        out11 = self.layer6(out10)

        out12 = self.up2(out11)
        out12 = crop_tensor(out5, out12)
        out12 = torch.cat((out12, out5), dim=1)
        out13 = self.layer7(out12)
        
        out14 = self.up3(out13)
        out14 = crop_tensor(out3, out14)
        out14 = torch.cat((out14, out3), dim=1)
        out15 = self.layer8(out14)
        
        out16 = self.up4(out15)
        out16 = crop_tensor(out1, out16)
        out16 = torch.cat((out16, out1), dim=1)
        out17 = self.layer9(out16)
        
        out18 = self.layer10(out17)

        return out18

# model = convnn(num_classes=1) 
# print(model)

# image = train_dataset[0]
# output = model(image)
# print(output.shape)  # Should output: torch.Size([1, 1, 388, 388]) based on U-Net original paper


In [ ]:
q=iter(train_loader)
x,y = next(q)
x = x.to(device)
y = y.to(device)

In [ ]:
model = convnn()
model.to(device)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.001, weight_decay=0.005,) #momentum = 0.9)

total_step = len(train_loader)

In [ ]:
criterion(model(x),y)

In [ ]:
for epoch in range(50):
    for i,(images,labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], LOss: {:.4f}'.format(epoch+1,50,loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(images)
        predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == y).sum().item()

    print('Accuracy of the network on the {} train images: {} %'.format(70, 100 * correct / total))

In [ ]:
w=iter(test_loader)
x,y = next(w)
x = x.to(device)
y = y.to(device)

In [ ]:
p=model(x)

In [ ]:
p.shape

In [ ]:
image_tensor = p
image_tensor = image_tensor.squeeze(0)
image_tensor = image_tensor.detach().cpu()
image_np = image_tensor.numpy()
image_np = np.transpose(image_np,(1,2,0))
plt.imshow(image_np)
plt.axis('off')
plt.show()